In [1]:
import heapq
import random
import time

class Order:
    def __init__(self, order_id, side, price, qty, timestamp, is_market=False):
        self.order_id = order_id
        self.side = side  # 'buy' or 'sell'
        self.price = price
        self.qty = qty
        self.timestamp = timestamp
        self.is_market = is_market

    def __lt__(self, other):
        if self.price == other.price:
            return self.timestamp < other.timestamp
        return self.price > other.price if self.side == 'buy' else self.price < other.price

class OrderBook:
    def __init__(self):
        self.buy_orders = []
        self.sell_orders = []
        self.order_id = 0
        self.trade_log = []

    def add_order(self, side, price, qty, is_market=False):
        self.order_id += 1
        timestamp = time.time()
        order = Order(self.order_id, side, price, qty, timestamp, is_market)
        self.match(order)

    def match(self, order):
        if order.side == 'buy':
            target_book = self.sell_orders
            opposite_side = 'sell'
        else:
            target_book = self.buy_orders
            opposite_side = 'buy'

        while target_book and order.qty > 0:
            best_opposite = target_book[0]
            if not order.is_market and (
                (order.side == 'buy' and order.price < best_opposite.price) or
                (order.side == 'sell' and order.price > best_opposite.price)
            ):
                break

            traded_qty = min(order.qty, best_opposite.qty)
            trade_price = best_opposite.price

            order.qty -= traded_qty
            best_opposite.qty -= traded_qty

            self.trade_log.append((order.side, trade_price, traded_qty, time.time()))

            if best_opposite.qty == 0:
                heapq.heappop(target_book)

        if order.qty > 0 and not order.is_market:
            heapq.heappush(
                self.buy_orders if order.side == 'buy' else self.sell_orders,
                order
            )

    def best_bid_ask(self):
        best_bid = self.buy_orders[0].price if self.buy_orders else None
        best_ask = self.sell_orders[0].price if self.sell_orders else None
        return best_bid, best_ask

    def print_order_book(self):
        print("\n--- Order Book ---")
        print("Buy Orders:")
        for order in sorted(self.buy_orders, reverse=True):
            print(f"{order.qty} @ {order.price}")
        print("Sell Orders:")
        for order in sorted(self.sell_orders):
            print(f"{order.qty} @ {order.price}")

    def print_trades(self):
        print("\n--- Trade Log ---")
        for trade in self.trade_log:
            print(f"{trade[0]} {trade[2]} @ {trade[1]}")

# Simulate market
def simulate_market():
    book = OrderBook()

    # Seed liquidity
    for _ in range(5):
        book.add_order('buy', round(random.uniform(98, 99), 2), random.randint(1, 5))
        book.add_order('sell', round(random.uniform(101, 102), 2), random.randint(1, 5))

    # Simulated HFT behavior
    for _ in range(20):
        side = random.choice(['buy', 'sell'])
        best_bid, best_ask = book.best_bid_ask()

        if side == 'buy':
            price = best_ask - 0.01 if best_ask else 100
        else:
            price = best_bid + 0.01 if best_bid else 101

        # Randomly place limit or market order
        is_market = random.random() < 0.3
        book.add_order(side, price, random.randint(1, 3), is_market=is_market)

    book.print_order_book()
    book.print_trades()

simulate_market()



--- Order Book ---
Buy Orders:
4 @ 98.48
1 @ 98.52
3 @ 98.64
1 @ 98.76
2 @ 101.19999999999999
3 @ 101.19999999999999
3 @ 101.19999999999999
1 @ 101.19999999999999
Sell Orders:
2 @ 101.21
2 @ 101.21
5 @ 101.26
1 @ 101.39
3 @ 101.74
1 @ 101.8

--- Trade Log ---
sell 2 @ 98.94
buy 3 @ 98.95
sell 2 @ 98.94
sell 1 @ 98.94
sell 2 @ 98.76
sell 1 @ 98.76
buy 2 @ 101.21
sell 1 @ 101.19999999999999
sell 1 @ 101.19999999999999
buy 3 @ 101.21
buy 1 @ 101.21
buy 1 @ 101.21
